In [1]:
import time
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

In [2]:
daily_demands = {'1': [(42.1,7.6),(41.9,12),(40.2,12.3),(37.2,12.3),(36.1,12.3),(34.6,12.6),(34.3,12.7)],#1096 4k
                '2': [(15.6,4.8),(15.3,7.9),(15.8,7.7),(14.8,8.1),(16.2,7.7),(15.9,7.8),(17.2,7.7)],#2177 3k
                '3': [(30,8.2),(29.3,12.3),(27.2,12.9),(26,14),(25.9,14.1),(25.6,14.4),(26.1,14.3)],#2189 4k
                '4': [(3.1,1.6),(2.8,2.5),(3,2.5),(2.5,2.6),(2.8,2.6),(2.7,2.6),(3.2,2.5)], # 3328 2k
                '5': [(7.9,3.1),(8.4,4.8),(7.9,4.8),(7.3,5),(7.4,5),(6.9,5),(8.1,5)], # 3349 2k
                '6': [(22.8,5.8),(21.4,9),(19.9,9.1),(16.4,10.1),(20.4,10.2),(20.1,10.4),(16.6,10.4)],#3889 3k
                '7': [(3.1,1.5),(2.9,2.5),(3.6,2.5),(3.1,2.4),(5,2.4),(5,2.5),(4.1,2.4)], #4388 1k
                '8': [(3.8,2.4),(4.2,3.8),(4,3.7),(4,4.1),(3.4,4),(3.4,3.9),(3.4,3.8)], #4816 1k
                '9': [(7.9,3.9),(8.7,5.6),(9,5.6),(9.1,6),(9.4,6),(9.4,6),(9.3,5.9)]} #5136 3k

In [3]:
store_grades = {'1': 4, '2': 3, '3': 4, '4': 2, '5': 2, '6': 3, '7': 1, '8': 1, '9': 3}

In [4]:
current_inventory={}
total=0
for store, demand in daily_demands.items():
    total =total+(sum(x[0] for x in demand))


weight=0
for k in store_grades.values():
    weight=weight+k
    
for i in daily_demands.keys():
    current_inventory[i]=math.ceil((int(total)*(store_grades[i]/weight)))

In [5]:
initial_inventory = current_inventory.copy()

In [6]:
def reset_inventory():
    return initial_inventory.copy()

In [7]:
num_stores = 9
current_day = 0

In [8]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [9]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [10]:
p_value=(costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio=norm.ppf(p_value)

In [11]:
current_day = 0
sales_dff = pd.DataFrame(columns=['Store', 'Day', 'Sales','Lost Sales','Demand','Inventory'])

In [12]:
def update_inventory(current_day):
    global sales_dff
    for i in current_inventory.keys():
        
        todays_sales = {i: max(0,int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])))}
        actual_sales = min(todays_sales[i], current_inventory[i])
        lost_sales = max(0, todays_sales[i] - current_inventory[i])
        current_inventory[i] = current_inventory[i]-actual_sales
        new_row = pd.DataFrame({'Store': [i], 'Day': [current_day], 'Sales': [actual_sales],'Lost Sales':lost_sales,'Demand':todays_sales[i],'Inventory':current_inventory[i]})
        sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)

    return current_inventory   

In [13]:
current_time=time.time()

results = pd.DataFrame()
cost_dict={}
for week in range(2000):
    current_inventory = reset_inventory()
    sales_dff = pd.DataFrame()

    for day in range(7):
        daily_inventory=update_inventory(day)

    weekly_cost=(sales_dff["Lost Sales"].sum()*costs["stockout"])+(sales_dff["Inventory"].sum()*costs["holding"])   
    cost_dict[week+1]=weekly_cost
    sales_dff["Week"]=week+1
    sales_dff["Cost"]=weekly_cost
    results = pd.concat([results,sales_dff], ignore_index=True)
    
print("Maximum cost for 100 week:",results["Cost"].max())
print("minimum cost for 100 week:",results["Cost"].min())    

Maximum cost for 100 week: 12702
minimum cost for 100 week: 4196


In [14]:
results["Lost Sales"].sum()/2000

167.131

In [15]:
results["Inventory"].sum()/2000

3047.847

In [16]:
unique_week_costs = results.drop_duplicates(subset="Week")[["Cost"]]
mean_cost = unique_week_costs["Cost"].mean()
print(mean_cost)

8061.777
